# Loading DAVIS recordings with events and frames

In [1]:
import tonic

sensor_size = tonic.datasets.DAVISDATA.sensor_size
frame_transform = tonic.transforms.ToFrame(sensor_size=sensor_size, time_window=25000) 
dataset = tonic.datasets.DAVISDATA(save_to='./data',
                                   recording='shapes_6dof'
#                                    transform=frame_transform
)

events, imu, images, target = dataset[0]

Using downloaded and verified file: ./data/davis_dataset/shapes_6dof.bag


In [3]:
events["t"]

array([       0,      494,      648, ..., 59735396, 59735397, 59735406])

In [6]:
images["ts"]-images["ts"][0]

array([0.00000000e+00, 4.40649986e-02, 8.81299973e-02, ...,
       5.96204410e+01, 5.96645060e+01, 5.97085719e+01])

In [7]:
len(images["ts"])


1356